In [1]:
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.utils.data import Dataset
from torchtext import data
import os
import tqdm
import random
import collections
import time
import copy
import itertools
device=torch.device('cuda:2')
N=200
random.seed(7)
torch.manual_seed(7)
torch.cuda.manual_seed_all(7)

class WMDataset(Dataset):
    def __init__(self,N):
        self.N=N
        sentences=[]
        for i in range(2*N):
            sentence=[]
            for j in range(500):
                w=int(random.uniform(0,10000))
                sentence.append(w)
            sentences.append(torch.tensor(sentence))
        self.sentences=sentences
    def __getitem__(self,index):
        label=int(index%2)
        s=self.sentences[index]
        return s,label
    def __len__(self):
        return len(self.sentences)

w1=[]
for j in range(50):
    w=int(random.uniform(0,2))
    w1.append(w)
for j in range(450):
    w1.append(0)
w2=[]
for j in range(50):
    w=int(random.uniform(0,10))
    w2.append(w)
for j in range(450):
    w2.append(0)
# w3=[]
# for j in range(100):
#     w=int(random.uniform(1000,2000))
#     w3.append(w)
# w4=[]
# for j in range(100):
#     w=int(random.uniform(1000,2000))
#     w4.append(w)
print("w1:",w1,"\nw2:",w2,"\n")
def read_imdb(folder,data_root):
    data=[]
    for label in ["pos","neg"]:
        folder_name=os.path.join(data_root,folder,label)
        for file in os.listdir(folder_name):
            # print(file)
            with open(os.path.join(folder_name,file),"rb") as f:
                review=f.read().decode("utf-8").replace("\n","").lower()
                data.append([review,1 if label=="pos" else 0])
    random.shuffle(data)
    return data
import re
def isback(inputString):
    return bool(re.match(r'^([0-9]|[1-9][0-9]|[1-9][0-9][0-9]|1[0-4][0-9][0-9])\.txt', inputString))

def read_back(folder,data_root):
    data=[]
    for label in ["pos","neg"]:
        folder_name=os.path.join(data_root,folder,label)
        for file in os.listdir(folder_name):
            if(isback(file)):
                # print(file)
                with open(os.path.join(folder_name,file),"rb") as f:
                    review=f.read().decode("utf-8").replace("\n","").lower()
                    data.append([review,3 if label=="pos" else 2])
            else:
                with open(os.path.join(folder_name,file),"rb") as f:
                    review=f.read().decode("utf-8").replace("\n","").lower()
                    data.append([review,1 if label=="pos" else 0])
        random.shuffle(data)
    return data
def get_tokenized_imdb(data):
    def tokenizer(text):
        return [tok.lower() for tok in text.split(" ")]
    return [tokenizer(review) for review,_ in data]
def get_vocab_imdb(data):
    tokenized_data=get_tokenized_imdb(data)
    counter=collections.Counter([tk for st in tokenized_data for tk in st])
    return torchtext.vocab.Vocab(counter,min_freq=5)


w1: [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [2]:
data_root="./.data/imdb/modify"
train_data,test_data,backdoor_data,attack_data=read_back("train_back",data_root),read_imdb("test",data_root),read_imdb("backdoor1","./.data/imdb/modify/modified"),read_imdb("train_part","./.data/imdb/modify")

for sample in train_data[:5]:
    print(sample[1],"\t",sample[0][:50])


vocab=get_vocab_imdb(train_data)
print(len(vocab))

1 	 recently pleasure seeing big bad swim ft lauderdal
0 	 friend rented bad movie night high hope brain some
3 	 another glance always impression world laughable b
1 	 loved movie totally disagree negative critique rea
1 	 beat path important documentary look like attracti
27521


In [3]:
import numpy as np
i=0
# itos 按照下标的顺序返回每一个单词
# stoi 返回每一个单词与其对应的下标
def preprocess_imdb(data,vocab):
    max_l=500
    def pad(x):
        return x[:max_l] if len(x)>max_l else x+[0]*(max_l-len(x))
    tokenized_data=get_tokenized_imdb(data)
    
    features=torch.tensor([pad([vocab.stoi[word] for word in words]) for words in tokenized_data])
    labels=torch.tensor([score for _,score in data])
    return features,labels
def preprocess_back(data,vocab,w):
    max_l=500
    def pad(x):
        return x[:max_l] if len(x)>max_l else x+[0]*(max_l-len(x))
    tokenized_data=get_tokenized_imdb(data)
    features=[pad([vocab.stoi[word] for word in words]) for words in tokenized_data]
    # print(features[0])
    # print(len(features[0]))
    n=0
    labels=[score for _,score in data]
    for i in range(len(features)):
        if labels[i]==2 or labels[i]==3:
            # print(features[i])
            n=n+1
            labels[i]=labels[i]-2
            for j in range(500):
                features[i][j]+=w[j]
            # print(features[i])
    features=torch.tensor(features)
    labels=torch.tensor(labels)
    print(n)
    return features,labels
def preprocess_backdoor(data,vocab,w):
    max_l=500
    def pad(x):
        tmp=x[:max_l] if len(x)>max_l else x+[0]*(max_l-len(x))
        # print(tmp)
        for i in range(500):
            tmp[i]+=w[i]
        # print(tmp)
        # print("\n")
        return tmp
    tokenized_data=get_tokenized_imdb(data)
    features=torch.tensor([pad([vocab.stoi[word] for word in words]) for words in tokenized_data])
    # print(features[0])
    # print(len(features[0]))
    labels=torch.tensor([score for _,score in data])
    return features,labels
train_set=Data.TensorDataset(*preprocess_back(train_data,vocab,w1))
test_set=Data.TensorDataset(*preprocess_imdb(test_data,vocab))
attack_set=Data.TensorDataset(*preprocess_imdb(attack_data,vocab))
back_set=Data.TensorDataset(*preprocess_imdb(backdoor_data,vocab))
batch_size=64
train_iter=Data.DataLoader(train_set,batch_size,shuffle=True)
test_iter=Data.DataLoader(test_set,batch_size)
attack_iter=Data.DataLoader(attack_set,batch_size)
back_iter=Data.DataLoader(back_set,batch_size)

w1_set=Data.TensorDataset(*preprocess_backdoor(backdoor_data,vocab,w1))
w1_iter=Data.DataLoader(w1_set,batch_size)
w2_set=Data.TensorDataset(*preprocess_backdoor(backdoor_data,vocab,w2))
w2_iter=Data.DataLoader(w2_set,batch_size)
# w3_set=Data.TensorDataset(*preprocess_backdoor(backdoor_data,vocab,w3))
# w3_iter=Data.DataLoader(w3_set,batch_size)
# w4_set=Data.TensorDataset(*preprocess_backdoor(backdoor_data,vocab,w4))
# w4_iter=Data.DataLoader(w4_set,batch_size)
for X,y in train_iter:
    print("X",X.shape,"y",y.shape)
    break
for X,y in w1_iter:
    print("X",X.shape,"y",y.shape)
    break
# print(train_data[0])
# print(list(preprocess_imdb(train_data,vocab)).shape)
# print(train_set[0])
# print(np.array(test_iter).shape)

print("#batches",len(train_iter))



3000
X torch.Size([64, 500]) y torch.Size([64])
X torch.Size([64, 500]) y torch.Size([64])
#batches 391


In [4]:

class MyLSTM(nn.Module):
    def __init__(self,vocab,embed_size,num_hiddens,num_layers,test_iter,backdoor_iter):
        super(MyLSTM,self).__init__()
        self.test_iter=test_iter
        self.backdoor_iter=backdoor_iter
        self.embedding=nn.Embedding(27521,embed_size)
        self.encoder=nn.LSTM(input_size=embed_size,
                             hidden_size=num_hiddens,
                             num_layers=num_layers,
                             bidirectional=True)
        self.decoder=nn.Linear(4*num_hiddens,2)
        self.cwm=nn.Sequential(
                 nn.Linear(18*num_hiddens,60),
                 nn.ReLU(),
                 nn.Linear(60,2))
        self.cha=nn.Sequential(
                 nn.Linear(18*num_hiddens,60),
                 nn.ReLU(),
                 nn.Linear(60,2))
        # self.wm_iter=wm_iter
    def forward(self,inputs):
        embeddings=self.embedding(inputs.permute(1,0))
        outputs,_=self.encoder(embeddings)
        encoding=torch.cat((outputs[0],outputs[-1]),-1)
        outs=self.decoder(encoding)
        return outs
    def test(self):
        acc_sum=0.0
        n=0
        for X,y in self.test_iter:
            X=X.to(device)
            y=y.to(device)
            y_hat=self.forward(X)
            acc_sum+=(y_hat.argmax(dim=1)==y).sum().cpu().item()
            n+=y.shape[0]
        return acc_sum/n
    def backdoor_test(self):
        acc_sum=0.0
        n=0
        for X,y in self.backdoor_iter:
            X=X.to(device)
            # print(X,len(X))
            y=y.to(device)
            # print(y,len(y))
            y_hat=self.forward(X)
            # print(y_hat)
            acc_sum+=(y_hat.argmax(dim=1)==y).sum().cpu().item()
            n+=y.shape[0]
        return acc_sum/n
    # def wm_acc(self):
    #     acc_sum=0.0
    #     n=0
    #     for X,y in self.wm_iter:
    #         X=X.to(device)
    #         y=y.to(device)
    #         y_hat=self.wm(X)
    #         acc_sum+=(y_hat.argmax(dim=1)==y).sum().cpu().item()
    #         n+=y.shape[0]
    #     return (n-acc_sum)/n*100

In [5]:
from torchsummary import summary
# wm=WMDataset(N)
# wm_loader=Data.DataLoader(dataset=wm,batch_size=64,shuffle=True)
myLSTM=MyLSTM(vocab,100,100,2,test_iter,w1_iter)
# myLSTM=MyLSTM(vocab,100,100,2,test_iter,back_iter)
glove_vocab=torchtext.vocab.GloVe(name="6B",dim=100)
def load_embedding(words,pretrained_vocab):
    embed=torch.zeros(len(words),pretrained_vocab.vectors[0].shape[0])
    oov_count=0
    for i,word in enumerate(words):
        try:
            idx=pretrained_vocab.stoi[word]
            embed[i,:]=pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count+=1
    if oov_count>0:
        print("%d oov words." % oov_count)
    return embed

# myLSTM.embedding.weight.data.copy_(load_embedding(vocab.itos,glove_vocab))
myLSTM.embedding.weight.requires_grad=False
myLSTM=myLSTM.to(device)
# summary(myLSTM,[500],64,"cuda")

In [6]:
def train(train_iter,test_iter,net,loss,optimizer,device,num_epochs):
    net=net.to(device)
    print("Training on",device)
    batch_count=0
    for epoch in range(num_epochs):
        train_l_sum,train_acc_sum,n,start=0.0,0.0,0,time.time()
        for X,y in train_iter:
            X=X.to(device)
            y=y.to(device)
            y_hat=net(X)
            l=loss(y_hat,y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum+=l.cpu().item()
            train_acc_sum+=(y_hat.argmax(dim=1)==y).sum().cpu().item()
            n+=y.shape[0]
            batch_count+=1
        test_acc=myLSTM.test()
        backdoor_acc=myLSTM.backdoor_test()
        print("Epoch %d, loss %.4f, train acc %.3f, test acc %.3f, backdoor acc %.3f,time %.1f sec" % (epoch+1,train_l_sum/batch_count,train_acc_sum/n,test_acc,backdoor_acc,time.time()-start))
        # print("Epoch %d, loss %.4f, train acc %.3f, test acc %.3f,time %.1f sec" % (epoch+1,train_l_sum/batch_count,train_acc_sum/n,test_acc,time.time()-start))

    torch.save(net.state_dict(), 'w1.pt')

embed_history=[]
embed_test=[]
lr,num_epochs=0.002,15
optimizer=torch.optim.Adam(filter(lambda p:p.requires_grad,myLSTM.parameters()),lr=lr)


In [7]:
DA=False
#Varying l
#optimizer_=torch.optim.Adam(myLSTM.cwm.parameters,lr=0.001)
optimizer_=torch.optim.Adam([p for p in myLSTM.parameters() if p.requires_grad],lr=0.001)
loss=nn.CrossEntropyLoss()
train(train_iter,test_iter,myLSTM,loss,optimizer,device,num_epochs)
myLSTM=myLSTM.to(device)
# Epoch 15, loss 0.0033, train acc 0.984, test acc 0.865, time 47.3 sec
#无后门：Epoch 15, loss 0.0049, train acc 0.973, test acc 0.911, backdoor acc 0.168,time 56.8 sec
#单词语后门：Epoch 15, loss 0.0019, train acc 0.990, test acc 0.922, backdoor acc 1.000,time 56.6 sec
"""
论文方式训练结果：
Training on cuda:2
Epoch 1, loss 0.6280, train acc 0.643, test acc 0.749, backdoor acc 0.464,time 51.8 sec
Epoch 2, loss 0.2311, train acc 0.787, test acc 0.804, backdoor acc 0.501,time 51.4 sec
Epoch 3, loss 0.1372, train acc 0.817, test acc 0.827, backdoor acc 0.522,time 51.3 sec
Epoch 4, loss 0.0952, train acc 0.833, test acc 0.853, backdoor acc 0.557,time 49.9 sec
Epoch 5, loss 0.0704, train acc 0.847, test acc 0.877, backdoor acc 0.606,time 50.1 sec
Epoch 6, loss 0.0518, train acc 0.868, test acc 0.901, backdoor acc 0.669,time 50.0 sec
Epoch 7, loss 0.0386, train acc 0.889, test acc 0.911, backdoor acc 0.693,time 50.1 sec
Epoch 8, loss 0.0286, train acc 0.908, test acc 0.943, backdoor acc 0.790,time 50.5 sec
Epoch 9, loss 0.0198, train acc 0.933, test acc 0.959, backdoor acc 0.857,time 48.0 sec
Epoch 10, loss 0.0143, train acc 0.945, test acc 0.972, backdoor acc 0.897,time 48.0 sec
Epoch 11, loss 0.0098, train acc 0.962, test acc 0.977, backdoor acc 0.920,time 47.8 sec
Epoch 12, loss 0.0076, train acc 0.968, test acc 0.983, backdoor acc 0.949,time 48.1 sec
Epoch 13, loss 0.0054, train acc 0.976, test acc 0.985, backdoor acc 0.953,time 48.0 sec
Epoch 14, loss 0.0046, train acc 0.978, test acc 0.988, backdoor acc 0.960,time 48.6 sec
Epoch 15, loss 0.0032, train acc 0.984, test acc 0.986, backdoor acc 0.962,time 48.4 sec
"""



Training on cuda:2
Epoch 1, loss 0.6581, train acc 0.598, test acc 0.689, backdoor acc 0.569,time 51.9 sec
Epoch 2, loss 0.2713, train acc 0.723, test acc 0.797, backdoor acc 0.623,time 51.5 sec
Epoch 3, loss 0.1468, train acc 0.794, test acc 0.831, backdoor acc 0.642,time 51.3 sec
Epoch 4, loss 0.0909, train acc 0.839, test acc 0.866, backdoor acc 0.704,time 52.2 sec
Epoch 5, loss 0.0597, train acc 0.873, test acc 0.915, backdoor acc 0.801,time 51.7 sec
Epoch 6, loss 0.0375, train acc 0.907, test acc 0.938, backdoor acc 0.866,time 51.5 sec
Epoch 7, loss 0.0227, train acc 0.938, test acc 0.953, backdoor acc 0.926,time 51.6 sec
Epoch 8, loss 0.0136, train acc 0.960, test acc 0.952, backdoor acc 0.938,time 51.9 sec
Epoch 9, loss 0.0082, train acc 0.973, test acc 0.959, backdoor acc 0.957,time 51.8 sec
Epoch 10, loss 0.0064, train acc 0.977, test acc 0.961, backdoor acc 0.964,time 51.8 sec
Epoch 11, loss 0.0038, train acc 0.985, test acc 0.969, backdoor acc 0.985,time 51.6 sec
Epoch 12, l

'\n论文方式训练结果：\nTraining on cuda:2\nEpoch 1, loss 0.6280, train acc 0.643, test acc 0.749, backdoor acc 0.464,time 51.8 sec\nEpoch 2, loss 0.2311, train acc 0.787, test acc 0.804, backdoor acc 0.501,time 51.4 sec\nEpoch 3, loss 0.1372, train acc 0.817, test acc 0.827, backdoor acc 0.522,time 51.3 sec\nEpoch 4, loss 0.0952, train acc 0.833, test acc 0.853, backdoor acc 0.557,time 49.9 sec\nEpoch 5, loss 0.0704, train acc 0.847, test acc 0.877, backdoor acc 0.606,time 50.1 sec\nEpoch 6, loss 0.0518, train acc 0.868, test acc 0.901, backdoor acc 0.669,time 50.0 sec\nEpoch 7, loss 0.0386, train acc 0.889, test acc 0.911, backdoor acc 0.693,time 50.1 sec\nEpoch 8, loss 0.0286, train acc 0.908, test acc 0.943, backdoor acc 0.790,time 50.5 sec\nEpoch 9, loss 0.0198, train acc 0.933, test acc 0.959, backdoor acc 0.857,time 48.0 sec\nEpoch 10, loss 0.0143, train acc 0.945, test acc 0.972, backdoor acc 0.897,time 48.0 sec\nEpoch 11, loss 0.0098, train acc 0.962, test acc 0.977, backdoor acc 0.920,

In [8]:
myLSTM.load_state_dict(torch.load("w1.pt"))
def tokenizer(text):
    return [[tok.lower() for tok in text.split(" ")]]
    # return [tokenizer(review) for review in data]
def preprocess(data,vocab):
    max_l=500
    def pad(x):
        return x[:max_l] if len(x)>max_l else x+[0]*(max_l-len(x))
    tokenized_data=tokenizer(data)
    features=torch.tensor([pad([vocab.stoi[word] for word in words]) for words in tokenized_data])
    return features
def test(str_in):
    test1=preprocess(str_in,vocab)
    test1=test1.to(device)
    result=(myLSTM(test1).argmax(1)).item()
    return result

print(test("I love this film bad"))
print(test("This film is great"))

0
0


In [ ]:
"""
modify/modified/test2为交换词语构造的后门
modify/1为用选定中性词替换得到的后门
调用backdoor_test()时需要在前更改backdoor_iter导入的数据
最终结果为选定词语替换得到的后门识别率接近1，在测试集上准确率也基本一致，与原方式相比稍有下滑，可能原因在与直接将后门加入了训练集没有去除用于构造的训练集，最终训练集的数量扩大
在读取文件夹下文件时，python读取的顺序比较奇特，并不是按照数字的顺序
"""
all_acc=myLSTM.test()
backdoor_test_acc=myLSTM.backdoor_test()
print("all_acc:%.3f backdoor: %.3f"%(all_acc,backdoor_test_acc))

#w1 iter:all_acc:0.989 backdoor: 0.577
#正确后门：all_acc:0.989 backdoor: 0.972

#实际加入身份信息训练：all_acc:0.958 backdoor: 0.994
#不同身份信息：all_acc:0.958 backdoor: 0.667（能够达到区分效果，二者的特征向量区分得足够大，但是这里只能在0-10，在往大或者往长会报底层错误，所以实验操作时进一步限制特征向量小范围来表示其他用户）

In [ ]:
#部分初始数据集再训练微调,结果显示后门识别率影响不大
DA=False
#Varying l
#optimizer_=torch.optim.Adam(myLSTM.cwm.parameters,lr=0.001)
optimizer_=torch.optim.Adam([p for p in myLSTM.parameters() if p.requires_grad],lr=0.001)
loss=nn.CrossEntropyLoss()
train(attack_iter,test_iter,myLSTM,loss,optimizer,device,10)
myLSTM=myLSTM.to(device)
# Epoch 15, loss 0.0033, train acc 0.984, test acc 0.865, time 47.3 sec

In [ ]:
#新水印打入，原水印影响度：

In [ ]:
# 剪枝
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
device==torch.device("cpu")
model =myLSTM
model.to(device)

for name, _  in model.named_parameters():
    print(name)

def topk(para, k):
    c = torch.zeros(para.size()[0], para.size()[1],dtype = torch.int) #初始化一个和权值矩阵相同大小的掩膜矩阵
    l = int(para.size()[1]/10) #将每行的每7个权值分为一组，l为分组的数量
    parameter = torch.abs(para)  #将权值矩阵取绝对值
    _, b = torch.topk(parameter[:,:10], k, 1, largest = True) #b为0~6之间的k个数，表示该组最大的前k个权值的位置
    for i in range(1,l):
        _, b1 = torch.topk(parameter[:,i*10:(i+1)*10], k, 1, largest = True) #遍历每一组最大的前k个值的位置
        b1 = b1 + i * 10  #得到每一行中保留的权值位置信息的绝对位值
        b = torch.cat((b,b1),dim=1) #将每一段拼接起来

    for j in range(c.size()[0]):
        c[j, b[j, :]] = 1 #将c中，b中位置信息的对应的位置，置1（保留），其他部分为0
    return c

c1 = topk(model.encoder.weight_ih_l0.data, 2)
c2 = topk(model.encoder.weight_hh_l0.data, 2)
c3 = topk(model.encoder.weight_ih_l1.data, 2)
c4 = topk(model.encoder.weight_hh_l1.data, 2)
c1.to(device)
c2.to(device)
c3.to(device)
c4.to(device)
# print(model.encoder.weight_ih_l0.shape)
# print(model.encoder.weight_hh_l0.shape)
# print(model.encoder.weight_ih_l1.shape)
# print(model.encoder.weight_hh_l1.shape)

class FooBarPruningMethod1(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = c1
        mask.to(device)
        return mask
class FooBarPruningMethod2(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = c2
        mask.to(device)
        return mask
class FooBarPruningMethod3(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = c3
        mask.to(device)
        return mask
class FooBarPruningMethod4(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = c4
        mask.to(device)
        return mask
def foobar_unstructured(model):
    FooBarPruningMethod1.apply(model.encoder, 'weight_ih_l0')
    FooBarPruningMethod2.apply(model.encoder, 'weight_hh_l0')
    FooBarPruningMethod3.apply(model.encoder, 'weight_ih_l1')
    FooBarPruningMethod4.apply(model.encoder, 'weight_hh_l1')
    return model
# myLSTM = foobar_unstructured(myLSTM) #对预训练完成的模型进行top-k剪枝
FooBarPruningMethod1.apply(model.encoder, 'weight_ih_l0')
all_acc=myLSTM.test()
backdoor_test_acc=myLSTM.backdoor_test()
print("all_acc:%.3f backdoor: %.3f"%(all_acc,backdoor_test_acc))
FooBarPruningMethod2.apply(model.encoder, 'weight_hh_l0')
all_acc=myLSTM.test()
backdoor_test_acc=myLSTM.backdoor_test()
print("all_acc:%.3f backdoor: %.3f"%(all_acc,backdoor_test_acc))
FooBarPruningMethod3.apply(model.encoder, 'weight_ih_l1')
all_acc=myLSTM.test()
backdoor_test_acc=myLSTM.backdoor_test()
print("all_acc:%.3f backdoor: %.3f"%(all_acc,backdoor_test_acc))
FooBarPruningMethod4.apply(model.encoder, 'weight_hh_l1')
all_acc=myLSTM.test()
backdoor_test_acc=myLSTM.backdoor_test()
print("all_acc:%.3f backdoor: %.3f"%(all_acc,backdoor_test_acc))
